## High-boosting filtering

Carlos Andrés Reyes Evangelista
157068
Ingeniería en Sistemas Computacionales


La presente práctica pretende demostrar la implementación del filtro conocido como High-boosting aplicado manualmente sobre una imagen.

In [1]:
## DECLARACIÓN DE VARIABLES Y MODULOS REQUERIDOS ##

import cv2 as cv
import numpy as np


from functools import reduce

#imagen original cargada desde disco
original = cv.imread('image.jpg')

#imagen en canal blanco y negro que será utilizada y sobrescrita por los algoritmos aquí presentados
bn = cv.cvtColor(original, cv.COLOR_BGR2GRAY)

#obtención de la cantidad de filas y columnas acordes a la dimensión de la imagen
rows, cols = bn.shape

#variable susceptible de ser cambiada, modificará directamente las máscaras en todo el programa
A = 1


mask1 = [-1, -1, -1, -1, (A + 8), -1, -1, -1, -1]


mask2 = [0, -1, 0, -1, (A + 4), -1, 0, -1, 0]

La mayor parte de esta práctica se llevará a cabo con la utilización de funciones provenientes del paradigma de programación funcional, tales como map, reduce y funciones lambda. La razón por la cual se decidió adquirir esta postura es por razones de práctica del paradigma y porque las operaciones a efectuar en esta práctica pueden verse más inherentemente apegadas al paradigma funcional. Se incluirá una explicación para cada operación para aumentar la claridad.

In [2]:
## FUNCIONES A UTILIZAR ##

def getNeighborhood (i, j):
    "Dada una coordenada (i, j) esta función regresa una lista de 9 tuplas con las coordenadas en la forma (x, y) de todos sus pixeles contiguos"
    #Utilizando la función zip se creará una nueva lista de tuplas combinando uno por uno los elementos de las siguientes listas:
    #[x x x] ([x] * 3) y [j-1, j, j+2] (range (j-1, j+2)). 
    #Esta operación se realizará para cada x que pertenece a {i-1, i, i+1}
    coordinates = map (lambda x: zip([x] * 3, range(j-1, j+2)), range(i-1, i+2))
    #esta instrucción simplemente convierte la lista de listas de tuplas en una sola lista de tuplas
    return [item for sublist in coordinates for item in sublist]


#Para la representación del padding decidí que no agregaría una hilera y una columna replicada para cada borde
#en su lugar decidí que, cada vez que una coordenada requerida estuviera fuera de las dimensiones de la imagen,
#sumaría 1 a la coordenada en x ó y si sale hacia valores negativos y restaría 1 si excede las dimensiones
#de esa manera la coordenada tomaría el valor de un pixel existente y contiguo a esa coordenada fuera de dimensiones
#Sin embargo, tampoco quería llenar de condicionales if innecesarios (if x < 0 x++, if y < 0 y++, if x > rows x--, if x = -1 and y = -1 then x = 0, y = 0)
#así que ingenié esta fórmula aritmética que simplemente añade una unidad si el valor es negativo o resta uno
#si el valor es positivo, así quedan cubiertos todos los casos pues se modulariza para x y para y de cada coordenada 
def addsubs1 (i):
    "Suma 1 si i < 0 y resta 1 si i > 0"
    if i != 0:
        i -= i / abs(i)
    return i


#Esta función implementa la anterior para cada componente de una coordenada dada
def correctOutOfBounds (tuple, rows, cols):
    "Dada una tupla que contiene coordenadas (x, y) y un límite de filas y columnas, retorna una tupla de coordenadas que se encuentran dentro de las dimensiones de la imagen"
    i, j = tuple
    
    if i not in range (rows):
        i = addsubs1(i)
    if j not in range (cols):
        j = addsubs1(j)
    return (int(i), int(j))


Cada una de las siguientes cajas contiene el código para cada una de las tres opciones de tratamiento de bordes solicitadas, preferentemente se sugiere correr una y brincar hasta la última celda para ver el resultado
(Ya mejoraré en este aspecto cuando tenga más dominio de Jupyter y Python que hasta ahora es casi nulo)

Las cajas siguientes utilizan una lógica similar con pequeñas variantes a depender del tratamiento de bordes utilizado, aquí explicaré el funcionamiento general y en cada caja comentaré las diferencias

Los for exteriores los dejé en aspecto imperativo para no sobrecargar en demasía con otro map anidado, recorren cada pixel con coordenadas (i, j)

currentNeighborhoodCoordinates = getNeighborhood(i, j) 
-> esta línea guarda en currentNeighborhoodCoordinates
una lista con las coordenadas de los vecinos para cada pixel en forma de tuplas

currentNeighborhood = map (lambda x: bn.item (x[0], x[1]), currentNeighborhoodCoordinates) 
-> esta línea guarda en currentNeighborhood el valor de la imagen (bn en este caso) para cada una de las coordenadas guardadas en la lista anterior, el resultado es una lista con los 9 valores de los pixeles vecinos incluyendo el actual a procesar

maskApplied         = map (lambda x, y: x * y, currentNeighborhood, mask1)
-> esta linea guarda en maskApplied el resultado de multiplicar cada valor de la lista anterior, 1 a 1, contra los valores de la máscara, el resultado es una lista con los valores ya multiplicados por la máscara

result              = reduce (lambda x, y: x + y, maskApplied) / 9
-> esta linea guarda en result el resultado final de haber hecho la sumatoria de los valores anteriores (ya multiplicados) y haberlo dividido entre 9

In [3]:
## FULL MASK LIMITING THE EXCURSIONS OF THE CENTER OF THE MASK TO BE IN THE IMAGE ##

#Para este método las iteraciones comienzan desde la segunda columna/fila y terminan en la penúltima de manera
#que cada coordenada (i, j) tiene vecinos como para efectuar su conversión

for i in range(1, rows-1):
    for j in range(1, cols-1):
        currentNeighborhoodCoordinates = getNeighborhood(i, j)
        currentNeighborhood = map (lambda x: bn.item (x[0], x[1]), currentNeighborhoodCoordinates)
        maskApplied         = map (lambda x, y: x * y, currentNeighborhood, mask1)
        result              = reduce (lambda x, y: x + y, maskApplied) / 9
        bn.itemset((i, j), bn.item(i, j) - result)

In [3]:
## USING A PARTIAL FILTER MASK AT THE IMAGE BORDERS ##"

#Este método varía del anterior en que esta vez las iteraciones se efectúan sobre la imagen completa y tiene
#diferencias en la instrucción:
#currentNeighborhood = map (lambda x: 0 if (x[0] < 0 or x[1] < 0 or x[0] >= rows or x[1] >= cols) else bn.item (x[0], x[1]), currentNeighborhoodCoordinates)
#donde, si alguna de las coordenadas guardadas en currentNeighborhoodCoordinates se sale de bounds
#el operador ternario retorna 0 como valor en lugar del valor actual del pixel en esa coordenada
#Esto funciona como máscara parcial porque un 0 en la sumatoria de valores no afectará al resultado final

for i in range (rows):
    for j in range (cols):
        currentNeighborhoodCoordinates = getNeighborhood (i, j)
        currentNeighborhood = map (lambda x: 0 if (x[0] < 0 or x[1] < 0 or x[0] >= rows or x[1] >= cols) else bn.item (x[0], x[1]), currentNeighborhoodCoordinates)
        maskApplied         = map (lambda x, y: x * y, currentNeighborhood, mask1)
        result              = reduce (lambda x, y: x + y, maskApplied) / 9
        bn.itemset((i, j), bn.item(i, j) - result)

In [3]:
## PADDING OF THE IMAGE BY REPLICATION ROWS AND COLUMNS ##

#Este método implementa los métodos de reemplazo de bordes inexistentes por los pixeles del borde contiguos
#explicados en una de las secciones anteriores.
#Esta lógica se aplica sobre las coordenadas obtenidas en currentNeighborhoodCoordinates = getNeighborhood (i, j)
# donde esta instrucción currentNeighborhoodCoordinatesCorrected = map (lambda x: x if (x[0] in range (rows) and x[1] in range (cols)) else correctOutOfBounds(x, rows, cols), currentNeighborhoodCoordinates)
#retorna un la misma coordenada si no sale de bounds o regresa la tupla corregida si es que sí sale de dimensiones

for i in range (rows):
    for j in range (cols):
        currentNeighborhoodCoordinates = getNeighborhood (i, j)
        currentNeighborhoodCoordinatesCorrected = map (lambda x: x if (x[0] in range (rows) and x[1] in range (cols)) else correctOutOfBounds(x, rows, cols), currentNeighborhoodCoordinates)
        currentNeighborhood = map (lambda x: bn.item (x[0], x[1]), currentNeighborhoodCoordinatesCorrected)
        maskApplied         = map (lambda x, y: x * y, currentNeighborhood, mask1)
        result              = reduce (lambda x, y: x + y, maskApplied) / 9
        bn.itemset((i, j), bn.item(i, j) - result)

In [4]:
cv.imshow("Original", original)
cv.imshow("High-boosted", bn)
cv.waitKey(0)
cv.destroyAllWindows()